In [14]:
from pyspark import SparkContext
sc = SparkContext(master="local[3]")

In [20]:
rdd1 = sc.parallelize(range(10),2)

In [25]:
print("Default parallelism: {}".format(sc.defaultParallelism))
print("Number of partitions: {}".format(rdd1.getNumPartitions()))
print("Partitioner: {}".format(rdd1.partitioner))
print("Partitions structure: {}".format(rdd1.glom().collect()))

Default parallelism: 3
Number of partitions: 2
Partitioner: None
Partitions structure: [[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]


In [23]:
rdd1.map(lambda x: x**x).collect()

[1, 1, 4, 27, 256, 3125, 46656, 823543, 16777216, 387420489]

In [17]:
words = ['Data','is','fun']

In [18]:
wordsRDD = sc.parallelize(words)

In [19]:
wordsRDD.reduce(lambda w,v: w if len(w) < len(v) else v)

'is'

In [26]:
sc.stop()